In [ ]:
import pandas as pd
import json
import csv
import io
import requests
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import json
from datetime import datetime
import time
import numpy as np
from shapely.geometry import Polygon
import geojson
from shapely.geometry import shape
from datetime import datetime

In [ ]:
os.chdir('C:/Users/jhuo/Documents/20220421_WABE_SB202/')

### 20220607 make 5 and 25 minute transit isochrones

Steps:<br>
1. generate isochrones
2. use county boundaries as bounding box

In [ ]:
boundary = gpd.read_file('tl_2020_us_county/tl_2020_us_county.shp')
GA = boundary[boundary.STATEFP == '13'].copy()
GA = GA.to_crs('EPSG:4326')
GA1 = GA.rename(columns = {'geometry':'county_boundary'}).iloc[:,[3,17]].copy()
GA1 = GA1.reset_index()

In [ ]:
allboxes = pd.read_csv('20220520_jan_21_jan_21_may_22_unified_edited.csv')

In [ ]:
makechrone = allboxes[(allboxes.in_2022 == True) | (allboxes.in_nov_2020 == True)]

In [ ]:
makechrone = makechrone.reset_index()

In [ ]:
# https://github.com/heremaps/here-location-services-python/blob/master/docs/notebooks/isoline_routing_restaurant_search.ipynb
os.environ["LS_API_KEY"] = "-"

In [ ]:
from here_location_services import LS
# from here_map_widget import Map, Marker, GeoJSON, Group
from here_location_services.config.isoline_routing_config import (
    RANGE_TYPE,
    ISOLINE_ROUTING_TRANSPORT_MODE,
)

In [ ]:
LS_API_KEY = os.environ.get("LS_API_KEY")
ls = LS(api_key=LS_API_KEY)

In [ ]:
datetime_object = datetime.strptime('2022-05-05T14:01:00', '%Y-%m-%dT%H:%M:%S')

In [ ]:
def get_driving_iso(df,i):
    lat = df.lat.iloc[i]
    long = df.long.iloc[i]
    
    listi = []
    for travel_time in [300,1500]:
        dicti = {}
        result1 = ls.calculate_isoline(
                    destination=[lat, long],
                    range=str(travel_time),
                    arrival_time=datetime_object,
                    range_type=RANGE_TYPE.time,
                    transport_mode=ISOLINE_ROUTING_TRANSPORT_MODE.car)
        data1 = result1.to_geojson()
        Polygon = shape(data1['features'][0]['geometry'])
        dicti['travel_time'] = travel_time
        dicti['lat'] = lat
        dicti['long'] = long
        dicti['geometry'] = Polygon
        listi.append(dicti)
    gdf = gpd.GeoDataFrame(listi)
    return(gdf)

In [ ]:
try1 = get_driving_iso(makechrone,0)

In [ ]:
alldf = gpd.GeoDataFrame()

In [ ]:
alldf = alldf.append(try1)

In [ ]:
for i in makechrone.index.tolist():
    try:
        df = get_driving_iso(makechrone,i)
        alldf = alldf.append(df)
        print (i)
    except Exception:
        print ('error')
        print (i)

In [ ]:
alldf.tail()

In [ ]:
alldf1 = alldf.copy()

In [ ]:
alldf1['dropbox_point'] = gpd.points_from_xy(alldf1.long, alldf1.lat, crs="EPSG:4326")
alldf1pts = alldf1.rename(columns = {'geometry':'isochrone','dropbox_point':'geometry'})
alldf1pts.crs = 'EPSG:4326'
# find out which county each drop box belongs to
alldf2 = gpd.sjoin(alldf1pts, GA, how = 'left', op = 'within')

In [ ]:
alldf2[pd.isna(alldf2.COUNTYFP) == True]

In [ ]:
alldf2 = alldf2.drop(['STATEFP', 'COUNTYFP', 'COUNTYNS', 'NAME', 'NAMELSAD', 'LSAD',
       'CLASSFP', 'MTFCC', 'CSAFP', 'CBSAFP', 'METDIVFP', 'FUNCSTAT', 'ALAND',
       'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry'], axis = 1)
alldf2 = alldf2.rename(columns = {'geometry':'drop_box_loc'})

In [ ]:
GA1 = GA.rename(columns = {'geometry':'county_boundary'}).iloc[:,[3,17]].copy()

In [ ]:
GA1 = GA1.reset_index()

In [ ]:
# GA1 = GA.rename(columns = {'geometry':'county_boundary'})[['GEOID','county_boundary']].copy()
alldf3 = pd.merge(alldf2, GA1, on = 'GEOID', how = 'outer')
alldf3 = alldf3.rename(columns = {'isochrone':'geometry'})

In [ ]:
alldf3 = alldf3.reset_index()

In [ ]:
clipped_isochrones = gpd.GeoDataFrame()

In [ ]:
for i in list(np.arange(0,len(alldf3))):
    try:
        gdf = gpd.GeoDataFrame(alldf3.iloc[[i]])
        gdf.crs ='EPSG:4326'
        clipping = gpd.GeoDataFrame([1], geometry=[(alldf3.county_boundary.iloc[i])], crs='EPSG:4326') 
        clipped = gpd.clip(gdf,clipping)
        clipped_isochrones = clipped_isochrones.append(clipped)
        print (i)
    except Exception:
        print('error')
        print(i)

In [ ]:
clipped_isochrones.info()

In [ ]:
clipped_isochrones = clipped_isochrones.rename(columns = {'travel_time':'travel_tim'})

In [ ]:
old_file = gpd.read_file('20220521_driving_isochrone_10_15_20_30.shp')

In [ ]:
old_file.info()

In [ ]:
new_file = pd.concat([old_file,clipped_isochrones])

In [ ]:
new_file = new_file.drop(['index','level_0','index_right'], axis = 1)

In [ ]:
new_file.info()

In [ ]:
new_file.lat = np.round(new_file.lat,6)
new_file.long = np.round(new_file.long,6)

In [ ]:
allboxesyr = allboxes[['lat','long','in_2022','in_nov_2020']].copy()

In [ ]:
allboxesyr.lat = np.round(allboxesyr.lat,6)
allboxesyr.long = np.round(allboxesyr.long,6)

In [ ]:
new_file1 = new_file.merge(allboxesyr, on =['lat','long'], how = 'outer')

In [ ]:
new_file1.info()

In [ ]:
allboxesyr[allboxesyr.lat == 31.292901]

In [ ]:
newfile1 = new_file1[~((pd.isna(new_file1.in_2022) == True) & (pd.isna(new_file1.in_nov_2020) == True))].copy()

In [ ]:
newfile1.info()

In [ ]:
newfile1.drop('county_boundary', axis = 1).to_file('.shp')